In [1]:
import pandas as pd
import contentful as ct
import numpy as np

from google.cloud import bigquery
from google.cloud import storage
from google_auth_oauthlib import flow
#import pandas_gbq
#from datalab.context import Context

In [2]:
#Initiate contentful clients
spaceID=''
accessToken=''

client = ct.Client(spaceID, accessToken, api_url='preview.contentful.com', timeout_s=10)
#SpaceID first, access token second
#Default time out is 1s, it's too short for queries

## Table of Content
* [Content Types](#Content-Types)
* [Content Entries](#Content-Entries)
    * [Offers](#Offers)
    * [Topics](#Topics)
    * [Experiences](#Experiences)
    * [Retail Categories](#Retail-Categories)
    * [Retailers](#Retailers)
    * [Unnest & Merge](#Unnest-&-Merge)
* [Appendix & Notes](#Appendix-&-Notes)

## Content Types

In [90]:
# new data frame with split value columns 
def get_content_types():
    
    content_types = client.content_types() #get content model from API
    ls_type=[] #empty list for content type
    ls_id=[] #empty list for ID
    
    for i in range(len(content_types)):
        ls_type.append(content_types[i].name)
        ls_id.append(content_types[i].id)
        
    df=pd.DataFrame({'ContentType':ls_type, 'ID':ls_id})
    
    return df

In [10]:
content_types = client.content_types()

In [91]:
df_ctypes=get_content_types()

In [92]:
df_ctypes

,ContentType,ID
0,Introductions,introductions
1,Featured Link,featuredLink
2,Call to Action,callToAction
3,Template,template
4,Store,store
5,Basic Content,basicContent
6,Offer,offer
7,Content Page,contentPage
8,Terms/Privacy Policy,terms
9,Automated Collection,automatedCollection


## Content Entries

### Offers
[Back to top](#Table-of-Content)

Can use the query below to check what fields are available under the "`offer`" content. Alternatively, view it directly from the Contentful backend.

In [41]:
#test_content=client.entry('dc74ef9eb7e04edcb1087b25d55376fc')
#test_content

In [116]:
#offers=client.entries({'content_type': 'offer'})
#offers[0].fields()

{'backend_title': 'Specsavers: Get two designer pairs no gap - AU',
 'title': 'Specsavers: Get two designer pairs no gap',
 'slug': 'get-two-designer-pairs-no-gap',
 'hero_image_set': <Entry[heroImageSet] id='2yo1QmYuEUFMvTsnCKiI0r'>,
 'short_description': 'We could all use a spare pair. Get two designer pairs from the $199 single-vision range, no gap. With selected health funds.',
 'body_copy': 'We could all use a spare pair. Get two designer pairs from the $199 single-vision range, no gap. With selected health funds.',
 'terms_and_conditions': '__Terms and Conditions__\nIn store only. Both pairs for same person. Includes single-vision lenses. Other lenses available at an extra cost. Subject to your\npolicy’s waiting periods, annual limits, fund rules and having the appropriate level of extras cover. Present your valid health fund\ncard to redeem in store.',
 'start_at': datetime.datetime(2020, 3, 3, 17, 0),
 'end_at': datetime.datetime(2020, 4, 25, 18, 0),
 'show_at': datetime.dateti

In [41]:
#get a sample of offers
#note spelling is slightly different from the field list

# sample_offers=client.entries({'content_type': 'offer', 
#                         'sys.createdAt[ge]': '2020-01-01',
#                                    'select': ['fields.backendTitle','fields.title','fields.bodyCopy',
#                                               'fields.experiences','fields.topics',
#                                              'fields.retailCategories'],
#                      'fields.title[exists]': True, #title has to exist
#                       'fields.reviewStatus': 'Submitted', #exclude null status, only unpublished content in Preview
#                                     'limit': 500, #default is 100, highest is 1000, but also has size restrictions
#                             })
            
# Other interesting things to look at:['fields.startAt','fields.endAt','fields.showAt','fields.hideAt','fields.stores'
#'include': 1, include 1 level of linked resource, this will make the result nested

In [134]:
#Function to get various content out of Contentful with pagination
    #limit size default is 100, highest is 1000, but also has size restrictions, so it's better to call 
                                                                        #multiple small batches to prevent failing
def get_content(type_list, limit_size):
    
    #initialise empty final df
    df_final=pd.DataFrame()
    
    for content in type_list: #loop through content type
        #initialize counters for pagination
        counter=0
        skip=0
        entry_array=np.array([])
        
        while True:

            entries=client.entries({'content_type': content, 
                               'sys.createdAt[ge]': '2020-01-01',
                                          'select': ['fields.backendTitle','fields.title','fields.bodyCopy',
                                            'fields.experiences','fields.topics','fields.retailCategories'],
                            'fields.title[exists]': True, #title has to exist
                             'fields.reviewStatus': 'Submitted', 
                                                            #exclude null status, use "Published" for Delivery API
                                           'limit': limit_size,                                     
                                            'skip': skip,
                                           'order':'sys.createdAt' #keep the sort order so as not to miss records
                             })
            counter=counter+1
            skip=counter*limit_size
            entry_array=np.append(entry_array,entries)
      
            if entries.total/skip < 1:
                break
    
    
        #initiate empty lists
        ls_id=[] 
        ls_title=[] 
        ls_backTitle=[] 
        ls_body=[] 
        ls_topic=[] 
        ls_experience=[] 
        ls_retailCat=[] 
    
        for i in range(len(entry_array)): #need to try catch each one as they have varying length
            try:          
                ls_title.append(entry_array[i].title)
            except:
                ls_title.append('')

            try:
                ls_backTitle.append(entry_array[i].backend_title)
            except:
                ls_backTitle.append('')

            try:
                ls_body.append(entry_array[i].body_copy)
            except:
                ls_body.append('')

            try:    
                ls_topic.append(entry_array[i].topics)
            except:
                ls_topic.append('')          

            try:    
                ls_experience.append(entry_array[i].experiences)
            except:
                ls_experience.append('')

            try:
                ls_retailCat.append(entry_array[i].retail_categories)
            except:
                ls_retailCat.append('')

            try:
                ls_id.append(entry_array[i].id)
            except:
                ls_id.append('')
        
        df=pd.DataFrame({'ID':ls_id, 'Title':ls_title, 'backendTitle':ls_backTitle, 'Body':ls_body,
                    'Topics':ls_topic, 'Experiences':ls_experience, 'retailCategories':ls_retailCat})
        
        df['ContentType']=content
    
        df_final=df_final.append(df)
        
    return df_final

In [135]:
df_content=get_content(type_list=['offer','news'], limit_size=500)

In [4]:
df_offers=get_content(type_list=['offer'], limit_size=500)

In [5]:
df_offers.head(3)

,ID,Title,backendTitle,Body,Topics,Experiences,retailCategories
0,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,[<Entry[topic] id='6nhn2f4DCgqYC6gkG4aWsW'>],[<Entry[experience] id='5dcYGiuzeguO6uoWWuo4o0...,[<Entry[retailCategory] id='5gmNNc76Y8ACoGSkkE...
1,5ewdl7y9P3EePJjtUCigbj,rebel: 25% off all women's clothing and footwear,rebel: 25% off all women's clothing and footwear,Make your Mother's Day with 25% off all women'...,,[<Entry[experience] id='7CThyLuCnCIMqqEcOKik0g'>],[<Entry[retailCategory] id='2Sm0gfks5igM4UMKOA...
2,5PlgcfFKch3HfkhUlWYQWC,David Jones: Mother's Day,David Jones: Mother's Day - AU,Save 20% on full-priced watches and jewellery ...,[<Entry[topic] id='6nhn2f4DCgqYC6gkG4aWsW'>],[<Entry[experience] id='5qszhP236wQugMyWKIEyaE...,[<Entry[retailCategory] id='2Sm0gfks5igM4UMKOA...


 The resulted table is nested with a list of IDs for `Topics`, `Experienes` and `Retail Categories`. Next step is then to extract the ID from the list, and join with the dimension table to get values.

### Topics
[Back to top](#Table-of-Content)

`Topics` are short descriptions for what a particular event, offer belongs to. This is often seasonal, or festivals based. A `topic` can be very broad (e.g. Summer) or very specific (e.g. Diwali). It can be centre-specific or it can apply all centres and countries. Currently there are roughly 69 `topics` tags.

In [6]:
#sample_topics = client.entries({'content_type': 'topic'})
#sample_topics[28].fields()
#suggest further look into backend_title, short_description, hero_image_set

In [7]:
def get_topics():
    
    entries = client.entries({'content_type': 'topic', 
                                    'select': ['fields.title','fields.topicType', 'fields.slug'],
                      'fields.title[exists]': True, #title has to exist
                                     'limit': 1000 #default is 100, highest is 1000, but also has size restrictions
                             })
    
    #initiate empty lists
    ls_id=[] 
    ls_title=[] 
    ls_type=[]
    ls_slug=[] 
    
    for i in range(len(entries)):
        try:          
            ls_title.append(entries[i].title)
            ls_type.append(entries[i].topic_type)
            ls_slug.append(entries[i].slug)
            ls_id.append(entries[i].id)
            
        except:
            pass
        
    df=pd.DataFrame({'ID':ls_id, 'Topic':ls_title, 'topicSlug':ls_slug, 'topicType':ls_type})
    df["topicType_Primary"] = df["topicType"].str[0]
    
    return df

In [8]:
df_topics=get_topics()

In [9]:
df_topics.head(6)

,ID,Topic,topicSlug,topicType,topicType_Primary
0,1rYjiIrcZZfhCFQu7X6351,Westfield Direct,westfield-direct,[Service],Service
1,2PlY6zT0oU4agsykg0Ki4k,Food,food,[Category],Category
2,6pvZpYVIZfnFjzjIneXk5Y,Your Westfield centre is open,covid-19,[Occasion / Moment],Occasion / Moment
3,1thOUehtgkyBqmEPKbUjPw,Your Westfield centre is open,covid-19,[Occasion / Moment],Occasion / Moment
4,2hlGaFsdSeABL8z9Sa4Kph,Taste. Shop. Play.,tasteshopplay,[Occasion / Moment],Occasion / Moment
5,129zZddW5ygoO6ecMqyc4o,Taste. Shop. Play.,tasteshopplay,[Occasion / Moment],Occasion / Moment


### Experiences
[Back to top](#Table-of-Content)

`Experiences` feature in the Navigation Menu. They are an idealised version of an in-centre precinct, and a great way for us to define and categorise our online offering. There are roughly 14 `experiences` tags currently.

In [10]:
#sample_experiences = client.entries({'content_type': 'experience'})
#sample_experiences[3].fields()

In [11]:
def get_experiences():
    
    entries = client.entries({'content_type': 'experience', 
                                    'select': ['fields.title','fields.shortDescription', 'fields.slug'],
                      'fields.title[exists]': True, #title has to exist
                                     'limit': 1000 #default is 100, highest is 1000, but also has size restrictions
                             })
    
    #initiate empty lists
    ls_id=[] 
    ls_title=[] 
    ls_desc=[] 
    ls_slug=[] 
    
    for i in range(len(entries)):
        try:          
            ls_title.append(entries[i].title)
            ls_desc.append(entries[i].short_description)
            ls_slug.append(entries[i].slug)
            ls_id.append(entries[i].id)
            
        except:
            pass
        
    df=pd.DataFrame({'ID':ls_id, 'Experience':ls_title, 'exSlug':ls_slug, 'exDescription':ls_desc})

    return df

In [12]:
df_experiences=get_experiences()

In [13]:
df_experiences.head(6)

,ID,Experience,exSlug,exDescription
0,6MpGLxDj20WOIEG8ISeSIG,Dine at Westfield,dining,Feeling hungry? Whether you're in the mood for...
1,5oxuyG6sCceMG4ysWws8wM,Women's fashion,womens-fashion,Make sun-drenched days your best friend with o...
2,6bFhxpYpc4aKmsKyWYAqOg,Fresh food,fresh-food,Get your weekly grocery shopping sorted with d...
3,5BMGIxwlGwG02wwGuMuSyy,Kids and family,kids-and-family,There’s so much for the kids to do at Westfiel...
4,1RwElb2xCkkOqI6MaK4i6c,Men's fashion,mens-fashion,"Explore your favourite men's fashion brands, i..."
5,1GkEVGBQzeyG88cI8WuYQw,Fashion,fashion,Explore your favourite men's and women's fashi...


### Retail Categories
[Back to top](#Table-of-Content)

`Retail Categories` offer a detailed way to define our retailers by what products and services they offer. This definition is then used to drive the online experience. There are currently 191 retail categories across a 3-level hierarchy.

In [14]:
#sample_rcat = client.entries({'content_type': 'retailCategory'})
#for i in range(10):
 #   print(i, sample_rcat[i].fields())

In [15]:
def get_retail_cat():
    
    entries = client.entries({'content_type': 'retailCategory', 
                                       'select': ['fields.title','fields.parentCategory', 'fields.level',
                                                   'fields.shortDescription'],
                         'fields.title[exists]': True, #title has to exist
                                        'limit': 1000 #default is 100, highest is 1000, but also has size restrictions
                                 })
    
    #initiate empty lists
    ls_id=[] 
    ls_title=[] 
    ls_desc=[] 
    ls_level=[] 
    ls_parent=[]
    
    for i in range(len(entries)):
        try:          
            ls_title.append(entries[i].title)
        except:
            ls_title.append(None)
            
        try:
            ls_level.append(entries[i].level)
        except:
            ls_level.append(None)
            
        try:
            ls_parent.append(entries[i].parent_category.id)
        except:
            ls_parent.append(None)

        try:
            ls_desc.append(entries[i].short_description)
        except:
            ls_desc.append(None)        

        try:
            ls_id.append(entries[i].id)
        except:
            ls_id.append(None)
            

        #Some retail experiences don't have short description, here need to try catch each 
        #individually to have the same length on all the lists
            
    df=pd.DataFrame({'ID':ls_id, 'retailCategory':ls_title, 'catLevel':ls_level, 'catDescription':ls_desc, 
                     'catParentID': ls_parent})
    
    df=df[df['retailCategory'].notnull()] #exclude the nulls as the exist filter in the API doesn't work
    df.reset_index(drop=True, inplace=True)
    
    return df

In [16]:
df_retailCat=get_retail_cat()

In [17]:
df_retailCat.head(10)

,ID,retailCategory,catLevel,catDescription,catParentID
0,4u13JL5Xry0k6wga4UsyoE,Women's accessories,3.0,Women's accessories,HLMccrmkkmG6eMuESSCe
1,DeQGXiP5JuYqK4McMsGQk,Foreign currency exchange,1.0,None,None
2,775vKJXqcEuqISqS0Eyi0M,Australian designer,2.0,Australian designer brands,6MeAgwhepOKIUwEaAUqcIw
3,7Ai2ELqu76Im0uMKyGAsQI,International designer,2.0,International designer brands,6MeAgwhepOKIUwEaAUqcIw
4,5864xdAInuuO8wua84A2kW,Premium brands,2.0,Premium brands,6MeAgwhepOKIUwEaAUqcIw
5,1llq4vWoGwoaOWYeQIuImO,Luxury,2.0,Luxury brands,6MeAgwhepOKIUwEaAUqcIw
6,6MeAgwhepOKIUwEaAUqcIw,Luxury and premium,1.0,Luxury and premium brands,None
7,1wvXKxWWlacCK0YWss4IGC,Formal and bridal,3.0,Formal occasion and bridal,HLMccrmkkmG6eMuESSCe
8,5Q0Aw6bZVCusMuOK8EaaIM,Cinemas,2.0,Cinemas,2MKqnvHffa2gMW6SQaaSc8
9,1pmPyfs1eQqie0eGs2UQcS,Bowling,2.0,Bowling,2MKqnvHffa2gMW6SQaaSc8


### Retailers
[Back to top](#Table-of-Content)

`Retailer` provides a brief description of the retailer and the retail category it falls into. This is then presented as the storefront on the website.

In [3]:
sample_retailer = client.entries({'content_type': 'retailer'})
sample_retailer[3].fields()

{'backend_title': 'Go Vita - AU',
 'title': 'Go Vita',
 'slug': 'go-vita',
 'hero_image_set': <Entry[heroImageSet] id='ykjL8KkPba02EU2U04YO8'>,
 'retailer_logo_set': <Entry[retailerLogoSet] id='2uIni2j5li40iGKyMcO6Wu'>,
 'short_description': 'Go Vita',
 'long_description': 'When you walk into your local Go Vita store you are dealing with the owners. That’s why you receive the personal touch and are treated like part of the family. In every store Go Vita health specialists treat each person as an individual and work with you to find the best whole health solution for your lifelong wellness. Wherever you are on the path of good health from novice to enthusiast you will be treated with care, respect and honesty.',
 'country': 'Australia',
 'payment_methods': ['Westfield Giftcard'],
 'experiences': [<Entry[experience] id='5qszhP236wQugMyWKIEyaE'>],
 'retail_categories': [<Entry[retailCategory] id='1hsBJtwSSE8uKIaksGOy0U'>,
  <Entry[retailCategory] id='1w5wPLGoPWGwyKCkSYiwoM'>,
  <Entry[ret

In [26]:
def get_retailer(limit_size):
    
    #initialise empty final df
    df_final=pd.DataFrame()
    
    counter=0
    skip=0
    entry_array=np.array([])    
    while True:

        entries=client.entries({'content_type': 'retailer', 
                                          'select': ['fields.backendTitle','fields.title','fields.slug',
                                            'fields.country','fields.retailCategories'],
                            'fields.title[exists]': True, #title has to exist
                                           'limit': limit_size,                                     
                                            'skip': skip,
                                           'order':'sys.createdAt' #keep the sort order so as not to miss records
                             })
        counter=counter+1
        skip=counter*limit_size
        entry_array=np.append(entry_array,entries)
      
        if entries.total/skip < 1:
            break
    
    
        #initiate empty lists
    ls_id=[] 
    ls_title=[] 
    ls_backTitle=[] 
    ls_slug=[] 
    ls_country=[]  
    ls_retailCat=[] 
    
    for i in range(len(entry_array)): #need to try catch each one as they have varying length
        try:          
            ls_title.append(entry_array[i].title)
        except:
            ls_title.append('')

        try:
            ls_backTitle.append(entry_array[i].backend_title)
        except:
            ls_backTitle.append('')

        try:
            ls_slug.append(entry_array[i].slug)
        except:
            ls_slug.append('')    

        try:    
            ls_country.append(entry_array[i].country)
        except:
            ls_country.append('')

        try:
            ls_retailCat.append(entry_array[i].retail_categories)
        except:
            ls_retailCat.append('')

        try:
            ls_id.append(entry_array[i].id)
        except:
            ls_id.append('')
        
    df=pd.DataFrame({'ID':ls_id, 'Title':ls_title, 'backendTitle':ls_backTitle, 'Slug':ls_slug,
                    'Country':ls_country, 'retailCategories':ls_retailCat})

    df=df[df['Title']!=''] #remove rows that are missing title(still in draft)   
    return df

In [27]:
df_retailer=get_retailer(limit_size=500)

In [9]:
df_retailer.head(5)

,ID,Title,backendTitle,Slug,Country,retailCategories
0,1qkoJRVPksiGm6gki84cA,Herb Steam & Massage,Herb Steam & Massage,herb-steam-massage,Australia,[<Entry[retailCategory] id='2yeydGBj3uSuo6g6UE...
1,5GyNAsSOyI0KkcKokceoAC,Mujosh,Mujosh,mujosh,Australia,[<Entry[retailCategory] id='4u13JL5Xry0k6wga4U...
2,6RfrD6JsJyI6mWG2maoQG2,dusk,dusk,dusk,Australia,[<Entry[retailCategory] id='6rajN0lkpaQIGkWeWe...
3,64YE9RIPew2eYOOcygG08o,Suzanne Grae,Suzanne Grae,suzanne-grae,Australia,[<Entry[retailCategory] id='HLMccrmkkmG6eMuESS...
4,5HYB8VcMta4mwAUMekewWe,High Five Cafe,High Five Cafe,high-five-cafe,Australia,[<Entry[retailCategory] id='3hTEu4e6qAMq26ouMk...


### Unnest & Merge
[Back to top](#Table-of-Content)

In [18]:
# function to iterate through the array and grab the id of each element for a list of columns
def unnest_col(df, col_list):
    
    df_result=df #copy original df
    
    for col in col_list:
        
        col_flat = pd.DataFrame([[i, x.id] #grabbing the element's ID instead of itself, itself is still an CF object
                       for i, y in df_result[col].apply(list).iteritems() 
                           for x in y], columns=['originalIndex', col]) #expand each element in list
        col_flat = col_flat.set_index('originalIndex')
    
        df_result = df_result.drop(col, 1)
        df_result = df_result.merge(col_flat, left_index=True, right_index=True)
    
        df_result = df_result.reset_index(drop=True)
        
    return df_result

In [19]:
df_unnest=unnest_col(df=df_offers,col_list=['Topics','Experiences','retailCategories'])

In [20]:
df_unnest.head(6)

,ID,Title,backendTitle,Body,Topics,Experiences,retailCategories
0,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,5dcYGiuzeguO6uoWWuo4o0,5gmNNc76Y8ACoGSkkE84Ay
1,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,5dcYGiuzeguO6uoWWuo4o0,6GrORNsUBaC4oAEU6Wy0IM
2,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,7zYIFyXKg0cKeUew60uOGc,5gmNNc76Y8ACoGSkkE84Ay
3,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,7zYIFyXKg0cKeUew60uOGc,6GrORNsUBaC4oAEU6Wy0IM
4,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,5BMGIxwlGwG02wwGuMuSyy,5gmNNc76Y8ACoGSkkE84Ay
5,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,6nhn2f4DCgqYC6gkG4aWsW,5BMGIxwlGwG02wwGuMuSyy,6GrORNsUBaC4oAEU6Wy0IM


In [21]:
#merge results to get meaningful labels
def merge_and_select(df_base, col_keep, topic=True, experience=True, retailCat=True):
    
    df=df_base #make a copy of the original df
    
    if topic:
        df=df.merge(df_topics, how='left', left_on='Topics', right_on='ID', suffixes=('', '_t'))
    
    if experience:
        df=df.merge(df_experiences, how='left', left_on='Experiences', right_on='ID', suffixes=('', '_e'))
    
    if retailCat:
        df=df.merge(df_retailCat, how='left', left_on='retailCategories', right_on='ID',suffixes=('', '_r'))
        
    df=df[col_keep]
    df['Verbatim']=df['backendTitle']+' '+df['Body']
    
    return df

In [22]:
df_final= merge_and_select(df_base=df_unnest,
            col_keep=['ID','Title','backendTitle','Body','Topic','topicSlug','topicType_Primary',
                      'Experience','exSlug','exDescription','retailCategory','catLevel','catDescription'])

In [23]:
df_final.head(3)

,ID,Title,backendTitle,Body,Topic,topicSlug,topicType_Primary,Experience,exSlug,exDescription,retailCategory,catLevel,catDescription,Verbatim
0,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,Mother's Day,mothers-day,Occasion / Moment,Entertainment,entertainment,Let us entertain you! Whether it’s Christmas m...,Book stores,2.0,Book stores,"Dymocks: 2 for $30 - Garden City, Chermside Sp..."
1,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,Mother's Day,mothers-day,Occasion / Moment,Entertainment,entertainment,Let us entertain you! Whether it’s Christmas m...,"Books, stationery and gifts",1.0,"Books, stationery and gifts","Dymocks: 2 for $30 - Garden City, Chermside Sp..."
2,5K2B3UttLqrt5ylIxD5PMs,Dymocks: 2 for $30,"Dymocks: 2 for $30 - Garden City, Chermside",Spoil the one you love! 2 for $30. \nSee in st...,Mother's Day,mothers-day,Occasion / Moment,Home and tech,home-tech,From soft bed linens and throws to tech and ap...,Book stores,2.0,Book stores,"Dymocks: 2 for $30 - Garden City, Chermside Sp..."


In [24]:
df_final.to_csv('final_unnest_offers.csv', index=False)

## Appendix & Notes
[Back to top](#Table-of-Content)

This is used to upload table to BigQuery.

In [11]:
# RLiao account json credentials
# service_account_json = '/Users/pjanney/Accesses/scg-dai-sci-dev.json' this is for running VM to BQ calls
client_secrets_json = '/Users/rliao/Credentials/client_secret_394671187685-qjjdbmab2a7m413p1rgq8fuv8lv0iufi.apps.googleusercontent.com.json'

# project
project_id = 'scg-dai-sci-dev'
dataset_id = 'Playground_rliao'
# scopes
scopes = ['https://www.googleapis.com/auth/bigquery',
          'https://www.googleapis.com/auth/cloud-platform']

In [12]:
def authenticate_credentials(client_secrets_json, scopes):
    """Prepare credentials for authentication
    :rtype: credentials object
    """
    print('...authenticating user')
    appflow = flow.InstalledAppFlow.from_client_secrets_file(client_secrets_json, scopes)
    appflow.run_local_server()
    credentials = appflow.credentials
    # return
    return(credentials)

# authenticate user and get credentials
credentials = authenticate_credentials(client_secrets_json= client_secrets_json,
                                       scopes = scopes)

...authenticating user
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=394671187685-qjjdbmab2a7m413p1rgq8fuv8lv0iufi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=kkL1Id4VascxmE0FoUGiDUva5Mi2tb&access_type=offline


In [13]:
#client = bigquery.Client.from_service_account_json(service_account_json)
bcclient = bigquery.Client(project= project_id, credentials= credentials)
#bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(credentials=credentials)

In [14]:
def upload_to_bq(dataset_id, table_name, df): #takes dataset_id, how you want to name it in bq, df as argument
    df.to_gbq(dataset_id+'.'+table_name, 
                project_id, # Context.default().project_id,
                chunksize=1000, 
                if_exists='replace' #append,fail
            )

In [29]:
upload_to_bq(dataset_id=dataset_id, table_name='retailers_from_contentful', df=df_retailer)

5it [00:31,  6.23s/it]
